In [9]:
%env CUDA_VISIBLE_DEVICES=GPU-8868e167-e666-53c7-6c41-d8e83081f07e

env: CUDA_VISIBLE_DEVICES=GPU-8868e167-e666-53c7-6c41-d8e83081f07e


In [15]:
from rdkit import Chem
from rdkit.Chem import Descriptors
import pandas as pd

In [16]:
#data
df = pd.read_xml(r'/home/lero/idrive/cmac/Christopher Boyle/DrugBank 2025/full database.xml')

In [17]:
#function to calculate mol desc from smiles
def getMolDescriptors(mol, missingVal=None):
    results = {}
    for nm, fn in Descriptors._descList:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        results[nm] = val
    return results

In [18]:
#parse the XML to extract the SMILES strings
import xml.etree.ElementTree as ET

# Load the XML
tree = ET.parse(r'/home/lero/idrive/cmac/Christopher Boyle/DrugBank 2025/full database.xml')
root = tree.getroot()

# DrugBank XML uses namespaces, so we need to handle them
ns = {'db': 'http://www.drugbank.ca'}  # adjust if your XML uses a different namespace

smiles_list = []
drug_names = []

for drug in root.findall('db:drug', ns):
    name_elem = drug.find('db:name', ns)
    smiles_elem = drug.find('db:calculated-properties/db:property[db:kind="SMILES"]/db:value', ns)
    
    name = name_elem.text if name_elem is not None else None
    smiles = smiles_elem.text if smiles_elem is not None else None
    
    if smiles:  # only keep drugs with SMILES
        drug_names.append(name)
        smiles_list.append(smiles)

# Convert to DataFrame
df_smiles = pd.DataFrame({'Name': drug_names, 'SMILES': smiles_list})
df_smiles

,Name,SMILES
0,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,Goserelin,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
2,Gramicidin D,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
3,Desmopressin,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
4,Cetrorelix,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...
...,...,...
12308,Tulmimetostat,[H][C@@]1(CC[C@@H](CC1)N1CC(C1)OC)[C@@]1(C)OC2...
12309,Ibuzatrelvir,COC(=O)N[C@H](C(=O)N1C[C@@H](C[C@H]1C(=O)N[C@@...
12310,Cetyl oleate,CCCCCCCCCCCCCCCCOC(=O)CCCCCCC\C=C/CCCCCCCC
12311,Cetyl myristoleate,CCCCCCCCCCCCCCCCOC(=O)CCCCCCC\C=C/CCCC


In [19]:
descriptor_results = []

for index, row in df_smiles.iterrows():
    smiles = row['SMILES']
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            descriptors = getMolDescriptors(mol)
            descriptor_results.append(descriptors)
        else:
            # instead of None, append empty dict
            descriptor_results.append({})
    except Exception as e:
        print(f"Error processing SMILES '{smiles}': {e}")
        descriptor_results.append({})

# Convert the list of dictionaries into a DataFrame
descriptor_df = pd.DataFrame(descriptor_results)

# Concatenate the original DataFrame with the descriptor DataFrame
final_df = pd.concat([df_smiles, descriptor_df], axis=1)
final_df = final_df.drop(['SMILES'], axis=1)

final_df.to_csv(r'/home/lero/idrive/cmac/DDMAP/PCA_data/Drugbank_descriptors.csv', index=False)
print('done')


[13:54:32] Explicit valence for atom # 13 Cl, 5, is greater than permitted
[13:54:32] SMILES Parse Error: syntax error while parsing: OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]
[13:54:32] SMILES Parse Error: check for mistakes around position 84:
[13:54:32] C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O
[13:54:32] ~~~~~~~~~~~~~~~~~~~~^
[13:54:32] SMILES Parse Error: extra open parentheses while parsing: OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]
[13:54:32] SMILES Parse Error: check for mistakes around position 40:
[13:54:32] 1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2
[13:54:32] ~~~~~~~~~~~~~~~~~~~~^
[13:54:32] SMILES Parse Error: extra open parentheses while parsing: OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C